In [1]:
import sqlalchemy
from sqlalchemy.sql import text
import pandas as pd
import datetime
import json
import yaml
import requests
import bs4
import copy
from copy import deepcopy


%matplotlib inline  
%pylab inline
pylab.rcParams['figure.figsize'] = (20,12)

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.5/dist-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['copy', 'text']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


### Conecta ao banco

In [2]:
with open('server_config.yaml', 'r') as f:
        server = yaml.load(f)

host = server['host']
database = server['database']
user = server['user']
password = server['password']


url = 'postgresql://{}:{}@{}/{}'
url = url.format(user, password, host, database)
engine = sqlalchemy.create_engine(url)

#### Query Medidas provisorias

In [5]:
tipo_proposicao = 'MPV'
mpv = pd.read_sql_query("SELECT * FROM c_camdep.proposicoes_id WHERE nomeproposicao LIKE '{} %%';".format(tipo_proposicao), engine)


#### Query Projetos de Leis Ordinarias

In [18]:
tipo_proposicao = 'PL'
regime_tramitacao = 'Ordinária'
plo = pd.read_sql_query("SELECT * from c_camdep.camdep_proposicoes_id WHERE nomeproposicao LIKE '{} %%' AND regimetramitacao = '{}';".format(tipo_proposicao,regime_tramitacao), engine)

#### Query Legislaturas

In [17]:
casa = 'CD'
legs = pd.read_sql_query("SELECT * from c_congresso.legislaturas WHERE sigla_casa = '{}';".format(casa,regime_tramitacao), engine)

In [19]:
mpv.head(2)

,id,capnum,datacaptura,nomeproposicao,idproposicao,idproposicaoprincipal,nomeproposicaoorigem,tipoproposicao,tema,ementa,...,ufautor,partidoautor,dataapresentacao,regimetramitacao,ultimodespacho,apreciacao,indexacao,situacao,linkinteiroteor,apensadas
0,20970,25,2015-11-06 09:19:26,MPV 1/2001,36450,None,None,Medida Provisória,"Economia, Administração Financeira e Orçamentá...","Abre crédito extraórdinario, em favor do Minis...",...,None,None,2001-09-20,Urgência,None,Proposição Sujeita à Apreciação do Plenário,"Abertura de crédito, crédito extraordinário, M...",Tranformada no(a) Lei Ordinária 10296/2001,http://www.camara.gov.br/proposicoesWeb/prop_m...,None
1,21142,25,2015-11-06 09:19:26,MPV 3/2001,36706,None,None,Medida Provisória,Tributação,Estabelece normas para registro do resultado l...,...,None,None,2001-09-27,Urgência,None,Proposição Sujeita à Apreciação do Plenário,"Autorização, pessoa jurídica, registro, result...",Tranformada no(a) Lei Ordinária 10305/2001,http://www.camara.gov.br/proposicoesWeb/prop_m...,None


In [20]:
plo.head(2)

,id,capnum,datacaptura,nomeproposicao,idproposicao,idproposicaoprincipal,nomeproposicaoorigem,tipoproposicao,tema,ementa,...,ufautor,partidoautor,dataapresentacao,regimetramitacao,ultimodespacho,apreciacao,indexacao,situacao,linkinteiroteor,apensadas
0,2369,20,2015-11-06 01:10:58,PL 115/1999,15064,None,None,Projeto de Lei,"Economia, Administração Financeira e Orçamentá...",Isenta a operação de crédito que especifica do...,...,DF,PT,1999-02-25,Ordinária,DESPACHO INICIAL A CEIC CFT (MERITO E ARTIGO 5...,Proposição Sujeita à Apreciação Conclusiva pel...,"Concessão, isenção, (IOF), operação financeira...",MESA - Arquivada,http://www.camara.gov.br/proposicoesWeb/prop_m...,None
1,2370,20,2015-11-06 01:10:58,PL 117/1999,15065,None,None,Projeto de Lei,Viação e Transportes,Acrescenta parágrafo único ao art. 218 da Lei ...,...,RS,PDT,1999-02-25,Ordinária,Tendo em vista a vigência da Lei n.º 11.334 de...,Proposição Sujeita à Apreciação Conclusiva pel...,"Alteração, Código de Trânsito Brasileiro, dife...",MESA - Arquivada,http://www.camara.gov.br/proposicoesWeb/prop_m...,None


In [21]:
legs.head(2)

,id_legislatura,id_api,sigla_casa,numero_legislatura,data_inicio,data_fim,data_eleicao,data_captura,url_captura,api_captura
0,06cb580dda1d34aa3c31cc88edebd3fc,-1823,CD,-1823,1823-04-17,1823-11-12,None,2017-08-03 18:16:41.613460,https://dadosabertos.camara.leg.br/api/v2/legi...,CD
1,0470713a1fd3af57c5f63bdf3d6a9e5e,-1821,CD,-1821,1821-12-01,1822-12-31,None,2017-08-03 18:16:41.613587,https://dadosabertos.camara.leg.br/api/v2/legi...,CD


In [22]:
mpv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1670 entries, 0 to 1669
Data columns (total 23 columns):
id                       1670 non-null int64
capnum                   1670 non-null int64
datacaptura              1670 non-null datetime64[ns]
nomeproposicao           1670 non-null object
idproposicao             1670 non-null int64
idproposicaoprincipal    0 non-null object
nomeproposicaoorigem     0 non-null object
tipoproposicao           1670 non-null object
tema                     1635 non-null object
ementa                   1670 non-null object
explicacaoementa         572 non-null object
autor                    1670 non-null object
idecadastro              0 non-null object
ufautor                  0 non-null object
partidoautor             0 non-null object
dataapresentacao         1670 non-null datetime64[ns]
regimetramitacao         1670 non-null object
ultimodespacho           1165 non-null object
apreciacao               1670 non-null object
indexacao             

In [23]:
plo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107458 entries, 0 to 107457
Data columns (total 23 columns):
id                       107458 non-null int64
capnum                   107458 non-null int64
datacaptura              107458 non-null datetime64[ns]
nomeproposicao           107458 non-null object
idproposicao             107458 non-null int64
idproposicaoprincipal    2 non-null object
nomeproposicaoorigem     1492 non-null object
tipoproposicao           107458 non-null object
tema                     45983 non-null object
ementa                   107458 non-null object
explicacaoementa         31189 non-null object
autor                    107458 non-null object
idecadastro              50066 non-null float64
ufautor                  98779 non-null object
partidoautor             98539 non-null object
dataapresentacao         107458 non-null datetime64[ns]
regimetramitacao         107458 non-null object
ultimodespacho           47006 non-null object
apreciacao              

In [24]:
legs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 10 columns):
id_legislatura        57 non-null object
id_api                57 non-null int64
sigla_casa            57 non-null object
numero_legislatura    57 non-null int64
data_inicio           57 non-null object
data_fim              57 non-null object
data_eleicao          0 non-null object
data_captura          57 non-null datetime64[ns]
url_captura           57 non-null object
api_captura           57 non-null object
dtypes: datetime64[ns](1), int64(2), object(7)
memory usage: 4.5+ KB


In [41]:
plo[(plo['dataapresentacao'] > legs['data_inicio'][55]) & (plo['dataapresentacao'] < legs['data_fim'][55])]

,id,capnum,datacaptura,nomeproposicao,idproposicao,idproposicaoprincipal,nomeproposicaoorigem,tipoproposicao,tema,ementa,...,ufautor,partidoautor,dataapresentacao,regimetramitacao,ultimodespacho,apreciacao,indexacao,situacao,linkinteiroteor,apensadas
79437,245910,34,2015-11-07 21:10:16,PL 110/2011,491063,None,None,Projeto de Lei,Direito e Defesa do Consumidor,"Acrescenta o art. 2ºA à Lei nº 10.214, de 27 d...",...,GO,PP,2011-02-03,Ordinária,Atualização da tramitação do Projeto de Lei n....,Proposição Sujeita à Apreciação do Plenário,"Alteração, pagamento, boleto bancário, agência...",CCJC - Tramitando em Conjunto,http://www.camara.gov.br/proposicoesWeb/prop_m...,None
79439,245749,34,2015-11-07 21:10:16,PL 3/2011,490886,None,None,Projeto de Lei,Arte e Cultura,Declara o Movimento Hip Hop manifestação de cu...,...,PE,PT,2011-02-03,Ordinária,Apense-se a este o PL-6756/2013.Em razão desta...,Proposição Sujeita à Apreciação Conclusiva pel...,"Declaração, movimento, atividade cultural, ini...",MESA - Arquivada,http://www.camara.gov.br/proposicoesWeb/prop_m...,None
79440,245752,34,2015-11-07 21:10:16,PL 5/2011,490889,None,None,Projeto de Lei,"Agricultura, Pecuária e Pesca; Recursos Hídric...",Dispõe sobre a criação do Programa Nacional de...,...,MG,PT,2011-02-03,Ordinária,"Às Comissões de Agricultura, Pecuária, Abaste...",Proposição Sujeita à Apreciação Conclusiva pel...,"Criação, Programa Nacional de Produção de Bioc...",CFT - Pronta para Pauta,http://www.camara.gov.br/proposicoesWeb/prop_m...,None
79441,245754,34,2015-11-07 21:10:16,PL 6/2011,490891,None,None,Projeto de Lei,Meio Ambiente e Desenvolvimento Sustentável; T...,Institui benefício fiscal para a atividade de ...,...,MG,PT,2011-02-03,Ordinária,Às Comissões de Finanças e Tributação (Mérito...,Proposição Sujeita à Apreciação Conclusiva pel...,"Alteração, legislação tributária federal, base...",MESA - Arquivada,http://www.camara.gov.br/proposicoesWeb/prop_m...,None
79442,245755,34,2015-11-07 21:10:16,PL 7/2011,490892,None,None,Projeto de Lei,"Desporto e Lazer; Direitos Humanos, Minorias e...",Determina que os órgãos competentes construam ...,...,MG,PT,2011-02-03,Ordinária,Às Comissões de Turismo e Desporto; Segurida...,Proposição Sujeita à Apreciação Conclusiva pel...,"Obrigatoriedade, Ministério do Esporte, constr...",CFT - Aguardando Parecer,http://www.camara.gov.br/proposicoesWeb/prop_m...,None
79443,245756,34,2015-11-07 21:10:16,PL 8/2011,490893,None,None,Projeto de Lei,Desenvolvimento Urbano e Trânsito; Tributação,Concede isenção do Imposto sobre Produtos Indu...,...,MG,PT,2011-02-03,Ordinária,Apense-se à(ao) PL-376/2007. Proposição Sujeit...,Proposição Sujeita à Apreciação Conclusiva pel...,"Isenção fiscal, IPI, Prefeitura Municipal, aqu...",MESA - Retirado pelo Autor,http://www.camara.gov.br/proposicoesWeb/prop_m...,None
79444,245758,34,2015-11-07 21:10:16,PL 10/2011,490895,None,None,Projeto de Lei,Meio Ambiente e Desenvolvimento Sustentável; T...,"Altera a legislação tributária, concede benefí...",...,MG,PT,2011-02-03,Ordinária,Às Comissões de Finanças e Tributação (Mérito...,Proposição Sujeita à Apreciação Conclusiva pel...,"Alteração, legislação tributária federal, isen...",MESA - Retirado pelo Autor,http://www.camara.gov.br/proposicoesWeb/prop_m...,None
79445,245761,34,2015-11-07 21:10:16,PL 13/2011,490898,None,None,Projeto de Lei,Direito e Defesa do Consumidor,Dispõe os serviços de atendimento telefônico g...,...,MG,PT,2011-02-03,Ordinária,"Às Comissões de Ciência e Tecnologia, Comunic...",Proposição Sujeita à Apreciação Conclusiva pel...,"Obrigatoriedade, fornecedor, bens, serviço, ce...",MESA - Retirado pelo Autor,http://www.camara.gov.br/proposicoesWeb/prop_m...,None
79446,245763,34,2015-11-07 21:10:16,PL 15/2011,490900,None,None,Projeto de Lei,Educação; Meio Ambiente e Desenvolvimento Sust...,Dispõe sobre a inclusão da disciplina Educação...,...,MG,PT,2011-02-03,Ordinária,Apense-se à(ao) PL-3788/2008. Proposição Sujei...,Proposição Sujeita à Apreciação Conclusi

In [38]:
legs['data_inicio'][55]

datetime.date(2011, 2, 1)